# Datasets and experiments

Let's figure out how to use the datasets and experiments feature from LangSmith.

The main idea is that we have a series of datasets with an associated name, together with a function to evaluate the performance on them.
Then, we can evaluate a pipeline on a dataset and this will be considered an experiment.
The aggregated results of our algorithm according to the evaluation function can be checked in the UI.

# Datasets

There are multiple ways to create a dataset, both from the [UI](https://docs.langchain.com/langsmith/manage-datasets-in-application) and directly with the [SDK](https://docs.langchain.com/langsmith/manage-datasets-programmatically).
Clearly, we will use the SDK here.

Datasets can be created from multiple different sources, ranging from our recorded traces to simply uploading a csv file or pandas dataframe.
To keep this simple, we will create a small mock dataset by directly providing a list of values.

In [ ]:
examples = [
    {
        "inputs": {"question": "What is the largest mammal?"},
        "outputs": {"answer": "The blue whale"},
        # "metadata": {"source": "Wikipedia"},
    },
    {
        "inputs": {"question": "What do mammals and birds have in common?"},
        "outputs": {"answer": "They are both warm-blooded"},
    },
    {
        "inputs": {"question": "What are reptiles known for?"},
        "outputs": {"answer": "Having scales"},
    },
    {
        "inputs": {"question": "What's the main characteristic of amphibians?"},
        "outputs": {"answer": "They live both in water and on land"},
    },
]

Once we have the data in a structured format, we can create a client and upload it to LangSmith.

In [ ]:
from dotenv import load_dotenv

load_dotenv("../.env.secrets")

True

In [ ]:
from langsmith import Client

client = Client()
dataset_name = "Elementary Animal Questions"
dataset_description = "Questions and answers about animal phylogenetics."

# Create a dataset in the LangSmith API
dataset = client.create_dataset(dataset_name=dataset_name, description=dataset_description)

# Create examples in a dataset
client.create_examples(dataset_id=dataset.id, examples=examples)

{'example_ids': ['dce4ff6e-9a11-439c-be43-62f69ddeb7e3',
  '20d6b4f2-655f-47cd-9238-439d1b2cf658',
  '99ba2cc6-862b-465d-841b-cdb3f6814d66',
  '85eb143d-af38-4f11-8d27-9f00d2e10fb8'],
 'count': 4}

Now we can explore the datasets and their examples.

In [ ]:
[dataset.name for dataset in client.list_datasets()]

['Elementary Animal Questions']

In [ ]:
examples = list(client.list_examples(dataset_name=dataset_name))
for example in examples:
    print(f"Example {example.id}:")
    print(example.inputs)
    print(example.outputs)
    print("\n")

Example 20d6b4f2-655f-47cd-9238-439d1b2cf658:
{'question': "What's the main characteristic of amphibians?"}
{'answer': 'They live both in water and on land'}


Example 85eb143d-af38-4f11-8d27-9f00d2e10fb8:
{'question': 'What do mammals and birds have in common?'}
{'answer': 'They are both warm-blooded'}


Example 99ba2cc6-862b-465d-841b-cdb3f6814d66:
{'question': 'What is the largest mammal?'}
{'answer': 'The blue whale'}


Example dce4ff6e-9a11-439c-be43-62f69ddeb7e3:
{'question': 'What are reptiles known for?'}
{'answer': 'Having scales'}




Furthermore, we can edit the examples.

In [ ]:
to_edit = examples[0]
to_edit.inputs, to_edit.outputs

({'question': "What's the main characteristic of amphibians?"},
 {'answer': 'They live both in water and on land'})

In [ ]:
client.update_example(
    example_id=to_edit.id,
    #   inputs={"input": "updated input"},  # Do not change the question, only the answer
    outputs={"answer": "They can live both in water and on land"},
    metadata={"foo": "bar"},
    split="train",
)

{'example_ids': ['20d6b4f2-655f-47cd-9238-439d1b2cf658'], 'count': 1}

# Run an experiment

We need 3 main elements to run an experiment: data, a target function and a list of evaluators.
We have the data, let's now define the others.

## Define the function

The target function takes in a dictionary `example.inputs` and outputs a dictionary with the predicted outputs.

In [ ]:
from langchain.chat_models import init_chat_model
from langsmith import traceable

llm = init_chat_model(model="anthropic:claude-haiku-4-5-20251001")


@traceable
def question_answerer(inputs: dict) -> dict:
    instructions = "Answer any question given by the user with a short and concise sentence."
    messages = [
        {"role": "system", "content": instructions},
        {"role": "user", "content": inputs["question"]},
    ]
    result = llm.invoke(messages)
    return {"answer": result.content}

## Define the evaluators

The evaluators will measure the success in the task.
They take in the inputs, the predicted output and the reference output when available.

In [ ]:
from pydantic import BaseModel, Field


class Evaluation(BaseModel):
    is_correct: bool = Field(description="Whether the predicted answer is correct")


def is_correct(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    "Uses an LLM as a judge to determine if the predicted answer is correct based on the reference one."
    instructions = (
        "Given a question and its reference answer, assess if the predicted answer is correct."
    )
    messages = [
        {"role": "system", "content": instructions},
        {
            "role": "user",
            "content": f"<question>{inputs['question']}</question>\n<reference_answer>{reference_outputs['answer']}</reference_answer>\n<predicted_answer>{outputs['answer']}</predicted_answer>",
        },
    ]
    result = llm.with_structured_output(Evaluation).invoke(messages)
    return result.is_correct


def response_length(inputs: dict, outputs: dict, reference_outputs: dict) -> int:
    "Count the response length in characters."
    return len(outputs["answer"])

## Run the evaluation

In [ ]:
results = client.evaluate(
    question_answerer,
    data=dataset.name,
    evaluators=[is_correct, response_length],
    experiment_prefix="haiku baseline",  # optional: experiment name prefix
    description="Testing the system",  # optional: experiment description
    max_concurrency=4,  # optional: add concurrency
)

View the evaluation results for experiment: 'haiku baseline-cb4351c3' at:
https://smith.langchain.com/o/534c6700-030d-4852-b384-f32d1978ac2d/datasets/659ab2bc-1703-48e0-89ff-3c287fa568bf/compare?selectedSessions=aad2add4-dcc5-4490-bcc0-f0ee91eea621




4it [00:03,  1.26it/s]


Now we can go to LangSmith to evaluate the results!